In [ ]:
# !pip install torch

In [ ]:
# Regular imports
import matplotlib.pyplot as plt
import torch

from torch import nn

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

In [ ]:
data_path = "/kaggle/input/majortry6/labeled_data.csv"

In [ ]:
import numpy as np
import pickle
import pandas as pd
pd.options.mode.copy_on_write = True

data = pd.read_csv(data_path)
df = pd.DataFrame(data)
print(df.columns)
df.columns = ['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither',
       'classk', 'tweet']

In [ ]:
print(len(df))
print(df.head())
print(df.classk.unique())

In [ ]:
df["classk"] = df["classk"].map({2:1, 1:0, 0:0}) 
# label = 0 = hate
# label = 1 = not_hate

In [ ]:
print(df.head())

In [ ]:
print(df.classk.unique())
print(df.iloc[2,:].tweet)

In [ ]:
tweets = df.tweet
res = []

for i in range(len(tweets)):
    splitt = tweets[i].split(":")
    if len(splitt) == 1:
        res.append(splitt[0])
    else:
        res.append(splitt[-1])
        
tweets = res

print(tweets[2])

In [ ]:
hate_words = ['Moist', 'Cunt', 'Panties', 'Fuck', 'Hate', 'Nigger', 'Pussy', 'Ass', 'Motherfucker', 'Bitch', 'Damn']
not_hate_words = ['Love', 'Peace', 'Kindness', 'Happiness', 'Respect', 'Friendship',
'Appreciation', 'Hope', 'Encouragement', 'Support', 'Caring']

In [ ]:
from transformers import AutoTokenizer
from transformers import BertModel

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

tokenizer("I love Pokemon")

# tokenizer.decode(tokenizer("I love Pokemon")['input_ids'])

In [ ]:
model = BertModel.from_pretrained("bert-base-uncased")

inputs = tokenizer("Hello my dog is cute", return_tensors="pt")
print(inputs)

outputs = model(**inputs)

print(len(outputs.last_hidden_state[0])) # => no. of tokens + [CLS] + [SEP]
print(len(outputs.last_hidden_state[0][1])) # => hidden dim size
print(len(outputs.last_hidden_state[0][2])) # => hidden dim size

In [ ]:
temp = []
model = BertModel.from_pretrained("bert-base-uncased")

for word in hate_words:
    inputs = tokenizer(word, return_tensors="pt")
#     print(inputs)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:len(outputs.last_hidden_state[0])], dim=0)
    temp.append(outputs.tolist())

# temp = temp[1:]
# print(hate_space)
# print(len(temp)) # => no. of words = no. of rows
# print(len(temp[0])) # => no. of columns in each row = embedding vector size

# hate_space = temp.clone().detach().requires_grad_(True)
hate_space = torch.tensor(temp, requires_grad=True)

hate_space.is_leaf
# hate_space
# hate_space.requires_grad

In [ ]:
temp = []
model = BertModel.from_pretrained("bert-base-uncased")

# with torch.no_grad():
for word in not_hate_words:
    inputs = tokenizer(word, return_tensors="pt")
#     print(inputs)
    outputs = model(**inputs)
    outputs = torch.mean(outputs.last_hidden_state[0][1:len(outputs.last_hidden_state[0])], dim=0)
    temp.append(outputs.tolist())

# temp = temp[1:]
# print(hate_space)
# print(len(temp)) # => no. of words = no. of rows
# print(len(temp[0])) # => no. of columns in each row = embedding vector size


not_hate_space = torch.tensor(temp, requires_grad=True)

not_hate_space.is_leaf
# not_hate_space
# not_hate_space.requires_grad

In [ ]:
# Trial matmul

t1 = torch.randn(11, 768)
t2 = torch.randn(4, 23, 768)

print(torch.transpose(t2, 1, 2).shape)

torch.matmul(t1, torch.transpose(t2, 1, 2)).shape

In [ ]:
class Custom_Dataset(torch.utils.data.Dataset):
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        
    def __getitem__(self, index: int) -> (torch.Tensor, int): # => (X, y)
        
        X = self.data[index]
        y = self.labels[index]

        return (X, y)
    
    
    def __len__(self) -> int:
        
        return len(self.data)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tweets, list(df.classk), test_size=0.2, shuffle=True)

print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

In [ ]:
train_data = Custom_Dataset(data=X_train, labels=y_train)
test_data = Custom_Dataset(data=X_test, labels=y_test)

train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=1, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=1, shuffle=False)

In [ ]:
# next(iter(test_loader)) 

In [ ]:
class Model0(nn.Module):
    
    def __init__(self, model, tokenizer):
        super().__init__()
        self.tokenizer = tokenizer
        self.model = model
    
    def forward(self, X):
#         print(X)
        inputs = tokenizer(X, return_tensors="pt")
        inputs.to(device)
#         print(inputs)
        outputs = self.model(**inputs)
#         print(outputs.last_hidden_state.shape)
        outputs = outputs.last_hidden_state.transpose(1, 2)
#         print(outputs.shape)
        Psk_hate = torch.matmul(hate_space, outputs)
#         print(Psk_hate.shape)
#         print(Psk_hate[0])
        Psk_not_hate = torch.matmul(not_hate_space, outputs)
        return (Psk_hate, Psk_not_hate)

In [ ]:
model0 = Model0(model, tokenizer)

# X, y = next(iter(train_loader))
# Psk_hate, Psk_not_hate = model0(X)

In [ ]:
# print(len(Psk_hate)) # batch_size
# print(len(Psk_hate[0])) # num_tokens (or hidden dimension size if transpose used in Model0 forward())
# print(len(Psk_hate[0][1])) # hidden dimension size (or num_tokens if transpose used in Model0 forward())

In [ ]:
# SSM

# outputs.mean().shape

# print(Psk_hate.mean())
# print(Psk_not_hate.mean())

In [ ]:
# y_predict = torch.tensor([Psk_hate.mean(), Psk_not_hate.mean()])
# torch.argmax(y_predict) # => predict this in SSM

In [ ]:
# correct = 0
# total = len(y_test)

# true = []
# preds = []

# model0.to(device)

# hate_space = hate_space.to(device)
# not_hate_space = not_hate_space.to(device)

# # Psk_hate = Psk_hate.to(device)
# # Psk_not_hate = Psk_not_hate.to(device)

# with torch.inference_mode():
    
#     for i, batch in enumerate(test_loader):
# #         if i == 1:
# #             break
#         if i % 1000 == 0:
#             print(i)
#         X, y = batch
#         Psk_hate, Psk_not_hate = model0(X)
# #         print(Psk_hate.mean(dim=(1,2)).shape)
# #         Psk_hate = Psk_hate.mean(dim=(1,2))
# #         Psk_not_hate = Psk_not_hate.mean(dim=(1,2))
# #         y_predict = torch.tensor([torch.argmax(torch.tensor([Psk_hate[i], Psk_not_hate[i]])) 
# #                                   for i in range(Psk_hate.shape[0])])
#         y_predict = torch.argmax(torch.tensor([Psk_hate.mean().item(), Psk_not_hate.mean().item()]))
# #         print(y_predict.shape)
# #         y_predict = torch.argmax(y_predict)
# #         print(y)
# #         print(y_predict)
        
#         true.extend(y)
#         preds.append(y_predict)
        
#         cur = torch.eq(y_predict, y)
# #         print(cur)
        
#         correct += torch.sum(cur)
# #         print(torch.sum(cur))
        
# #         if y == y_predict:
# #             correct += 1
    
# print(f"Accuracy: {correct/total}")
# print(f"Accuracy: {torch.sum(torch.eq(torch.tensor(true), torch.tensor(preds))) / total}")

In [ ]:
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# cm = confusion_matrix(y_test, preds)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm)
                              
# disp.plot()
# plt.show()

In [ ]:
# from sklearn.metrics import f1_score

# print(f1_score(y_test, preds, average='macro'))

In [ ]:
# from sklearn.metrics import precision_score, recall_score

# print(precision_score(y_test, preds, average='macro'))
# print(recall_score(y_test, preds, average='macro'))

# Semi-SSM

In [ ]:
def L_inter(spaces):
    
    means = [torch.mean(spaces[i]) for i in range(spaces.shape[0])]
    
    loss = torch.tensor(0.0, requires_grad=True)
    
    for k in range(len(means)):
        cur = torch.tensor(0.0, requires_grad=True)
        for l in range(len(means)):
            if l == k:
                continue
            cur = cur + (1 / (1 - ((means[k] * means[l]))))
        
        loss = loss + cur
    
    return loss
            
            

def L_intra(spaces):
    loss=torch.tensor(0.0, requires_grad=True)
    for k in range(len(spaces)):
        loss=loss+1/torch.var(spaces[k], dim=0)
    
    return torch.sum(loss)/loss.shape[0]




In [ ]:
L_inter(torch.concat((torch.tensor([[0.33, 0.54, 0.37], [0.72, 0.74, 0.31], [0.61, 0.26, 0.56]], dtype=torch.float32), 
       torch.tensor([[0.22, 0.32, 0.15], [0.12, -0.08, 0.45], [0.14, 0.28, -0.11]], dtype=torch.float32))))

In [ ]:
# temp = torch.tensor(10.3, requires_grad=True)

# temp

In [ ]:
# res = [temp, temp]

# loss = 0

# loss += res[0] * res[1]

# loss

In [ ]:
hate_space.is_leaf

In [ ]:
not_hate_space.is_leaf

In [ ]:
correct = 0
total = len(y_train)

true = []
preds = []

model0.to(device)

optimizer = torch.optim.Adam(params=[hate_space, not_hate_space], lr=0.01)

hate_space = hate_space.to(device)
not_hate_space = not_hate_space.to(device)
    
for i, batch in enumerate(train_loader):
#         if i == 1:
#             break
    if i % 1000 == 0:
        print(i)
    X, y = batch
    Psk_hate, Psk_not_hate = model0(X)
#         print(Psk_hate.mean(dim=(1,2)).shape)
#         Psk_hate = Psk_hate.mean(dim=(1,2))
#         Psk_not_hate = Psk_not_hate.mean(dim=(1,2))
#         y_predict = torch.tensor([torch.argmax(torch.tensor([Psk_hate[i], Psk_not_hate[i]])) 
#                                   for i in range(Psk_hate.shape[0])])
    y_predict = torch.argmax(torch.tensor([Psk_hate.mean().item(), Psk_not_hate.mean().item()]))
#         print(y_predict.shape)
#         y_predict = torch.argmax(y_predict)
#         print(y)
#         print(y_predict)

    loss = L_inter(torch.concat((hate_space, not_hate_space)))+ L_intra(torch.stack([hate_space, not_hate_space]))
#     print(loss)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    true.extend(y)
    preds.append(y_predict)

    cur = torch.eq(y_predict, y)
#         print(cur)

    correct += torch.sum(cur)
#         print(torch.sum(cur))

#         if y == y_predict:
#             correct += 1
    
# print(f"Train Accuracy: {correct/total}")
# print(f"Train Accuracy: {torch.sum(torch.eq(torch.tensor(true), torch.tensor(preds))) / total}")

In [ ]:
torch.concat((hate_space, not_hate_space)).shape

In [ ]:
correct = 0
total = len(y_test)

true = []
preds = []

with torch.inference_mode():
    for i, batch in enumerate(test_loader):
        if i % 1000 == 0:
            print(i)
        X, y = batch
        Psk_hate, Psk_not_hate = model0(X)

        y_predict = torch.argmax(torch.tensor([Psk_hate.mean().item(), Psk_not_hate.mean().item()]))

        true.extend(y)
        preds.append(y_predict)

        cur = torch.eq(y_predict, y)

        correct += torch.sum(cur)

print(f"Test Accuracy: {correct/total}")
print(f"Test Accuracy: {torch.sum(torch.eq(torch.tensor(true), torch.tensor(preds))) / total}")        

In [ ]:
from sklearn.metrics import precision_score, recall_score

print(precision_score(y_test, preds, average='macro'))
print(recall_score(y_test, preds, average='macro'))

In [ ]:
input1 = torch.randn(100, 1)
input2 = torch.randn(100, 1)
cos = nn.CosineSimilarity(dim=1, eps=1e-6)
output = cos(input1, input2)
output